# DBS/TMS awake/asleep comparison- Consciousness

## Load in data

In [ ]:
import os,re,glob
import os.path as op
from MEEGbuddy import MEEGbuddy
from MEEGbuddy.bv2fif import bv2fif
import numpy as np
from mne import find_events, events_from_annotations
from mne.io import Raw
import matplotlib.pyplot as plt
from pandas import DataFrame

subject = 'PT'
session = '20190726'
ref_ch = None
perturbation = 'TMS'
event_id = 'Stimulus/S255' #'TMS' #'Response/R 15'

datarex = re.compile(r'[%s|%s]*.vhdr' % (subject.lower(), subject.upper()))
correx = re.compile(r'[%s|%s]*.csv' % (subject.lower(), subject.upper()))
    
if True:
    this_dir = op.join('data', perturbation, subject, session)
    files = os.listdir(this_dir)
    if not files:
        print('Subject data must be in a folder with their name')
    corf = None
    for f in files:
        if correx.search(op.basename(f)):
            corf = f
    for f in files:
        if all([datarex.search(f),not 'fif' in f,not 'avg' in f, 
                not '-TMS.vhdr' == op.basename(f)[-9:],not '-DBS.vhdr' == op.basename(f)[-9:]]):
            fdata = (op.join(this_dir, op.dirname(f),
                     op.basename(f).split('.')[0]+'-raw.fif'))
            behavior = op.join(this_dir, op.dirname(f),
                     op.basename(f).split('.')[0]+'.csv')
            if not op.isfile(fdata):
                bv2fif(op.join(this_dir, f), op.join(this_dir, corf),
                       ref_ch=ref_ch, dbs=perturbation.lower()=='dbs')
            raw = Raw(fdata, preload=False)
            events, event_id2 = events_from_annotations(raw)
            if not events.size > 0:
                events = find_events(raw)
            if len(np.unique(events[:, 2])) > 1:
                events = events[np.where(events[:,2] == events[1,2])[-1]] #skip new segment
            df = DataFrame({'Trial': range(1, events.shape[0] + 1), 'State': 'TBD'})
            df.to_csv(behavior, index=False)
            behavior_description = {'Trial': {'Description': 'The order of the stimulus presentations',
                                              'Baseline': {'Channel': event_id, 'Time Min': -0.4, 'Time Max': -0.01},
                                              'Events': {perturbation: {'Channel': event_id, 'Time Min': -0.5, 'Time Max': 1.0}},
                                              'Exclude Trials': [0, len(df) - 1]},
                                    'State': {'Description': 'Whether the subject was awake or asleep on ' + 
                                              'that trial'}}
            #
            fs_dir = op.join(os.getcwd(), 'data', perturbation)
            bemf = op.join(fs_dir,subject, 'bem', '%s-5120-5120-5120-bem-sol.fif' % (subject))
            srcf = op.join(fs_dir,subject, 'bem', '%s-oct-6-src.fif' % (subject))
            transf = op.join(fs_dir, subject, session, 'COR-%s_%s.fif' % (subject, session))
            #
            print('Loading in to MEEGbuddy %s %s' %(subject, session))
            data = MEEGbuddy(subject=subject, session=session, fdata=fdata,
                             task=perturbation + 'EEG', eeg=True,
                             subjects_dir=op.join(os.getcwd(), 'data'),
                             fs_subjects_dir=fs_dir,
                             bemf=bemf, srcf=srcf, transf=transf, behavior=behavior,
                             behavior_description=behavior_description)

Extracting parameters from ./JM/1_31_2018_JM_con_pilot.vhdr...
Setting channel info structure...
Reading 0 ... 99771499  =      0.000 ...  3990.860 secs...


# Reload MEEGbuddy

In [ ]:
from MEEGbuddy import loadMEEGbuddy
import os.path as op
import os

subject = 'PT'
session = '20190726'
perturbation = 'TMS'

data = loadMEEGbuddy(op.join(os.getcwd(), 'data'), subject, session,
                     eeg=True, task=perturbation + 'EEG')

# Save to BIDS

In [ ]:
data.save2BIDS('data_BIDS')

## Channel and Trial Rejection

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'

data.makeEpochs(keyword_out='Preprocessed', overwrite=True)
#data.filterEpochs(event, keyword_in='Preprocessed', l_freq=None, h_freq=40, overwrite=True)
data.plotEvoked(event, keyword='Preprocessed', ylim={'eeg':[-60, 40]},
                tmin=-0.5, tmax=0.5)
ok, first = False, True
while not ok:
    data.plotRaw(keyword_in=(None if first else 'Preprocessed'),
                 keyword_out='Preprocessed', l_freq=None, h_freq=None, overwrite=True)
    data.makeEpochs(keyword_in='Preprocessed', overwrite=True)
    #data.filterEpochs(event, keyword_in='Preprocessed', l_freq=None, h_freq=40, overwrite=True)
    data.plotEvoked(event, keyword='Preprocessed', ylim={'eeg':[-60,40]}, tmin=-0.5, tmax=0.5)
    ok, first = input('Type Y or y if ok, anything else to go back\n').lower() == 'y', False
plt.show()
#
ok = False
while not ok:
    raw_interp,_ = data.plotInterpolateArtifact(event, keyword='Preprocessed', mode='spline', 
                                                npoint_art=126, offset=25, tmin=-0.1, tmax=0.1, 
                                                ylim=[-1e-4,1e-4])
    plt.show()
    ok = input('Type Y or y if ok, anything else to go back\n').lower() == 'y'
#
data.applyInterpolation(raw_interp, keyword_out='Interpolated',
                        overwrite=True)
data.filterRaw(keyword_in='Interpolated', l_freq=1.0, overwrite=True)
data.makeEpochs(keyword_in='Interpolated', keyword_out='Preprocessed', overwrite=True)
data.downsample(event=event, keyword_in='Preprocessed', new_sfreq=200, overwrite=True)
#
ok = False
while not ok:
    data.plotEvoked(event, keyword='Preprocessed', 
                    ylim={'eeg':[-60, 40]}, tmin=-0.5, tmax=0.5)
    data.plotEpochs(event, keyword_in='Preprocessed', 
                    l_freq=None, h_freq=None, overwrite=True)
    ok = input('Type Y or y if ok, anything else to go back\n').lower() == 'y'

## ICA

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
# precompute ICA
data.findICA(event=event, keyword_in='Preprocessed', keyword_out='ICA',
             n_components=0.95, overwrite=True, overwrite_ica=True)
#
data.filterEpochs(event, h_freq=40, keyword_in='Preprocessed', keyword_out='Vis', overwrite=True)
data.plotEvoked(event, keyword='Vis', ylim={'eeg':[-100, 100]})
data.plotTopo(event, keyword='Vis')
ok = False
while not ok:
    data.plotICA(event=event, keyword_in='Preprocessed', keyword_out='ICA')
    data.filterEpochs(event, h_freq=40, keyword_in='ICA', keyword_out='Vis', overwrite=True)
    data.plotEvoked(event, keyword='Vis', ylim={'eeg':[-100, 100]})
    data.plotTopo(event, keyword='Vis')
    ok = input('Type Y or y if ok, anything else to go back\n').lower() == 'y'
plt.show()
data.filterEpochs(event, h_freq=80, keyword_in='ICA', overwrite=True)
epochs = data._load_epochs(event, keyword='ICA')
bl_epochs = epochs.crop(tmin=data.baseline[1], tmax=data.baseline[2])
data._save_epochs(bl_epochs, 'Baseline', keyword='ICA')

## Multitaper Spectrogram and Sleep/Wake Assignment

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
state_times = data.psdMultitaper(keyword='Preprocessed', overwrite=False, ch='Oz', vmax=-10)
data.assignConditionFromStateTimes(event, state_times)

## Clean Up

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
data.remove(keyword='Preprocessed')
data.remove(event, keyword='Preprocessed')
data.remove(keyword='Interpolated')

## Source Space

In [3]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
data.epochs2source(event, 'State', method='MNE', snr=1.0, 
                       keyword_in='ICA', keyword_out='Clean', 
                       overwrite=True)



  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 251.45it/s]

### Plot Source Space (optional)

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
data.plotSourceSpace(event,'State', keyword='Clean', tmin=-0.1, tmax=0.5)
plt.close('all')

## PCI

The interesting part!

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
if not data._has_epochs(event, keyword='PCI'):
    values = [value for value in data._default_values('State') if not value == 'Neither']
    alphas = [0.01] if event == 'TMS' else [0.01, 0.1, 0.5]
    if not all([data._has_noreun_PCI(event, 'State', value, keyword='alpha=%s' %(alpha)) 
                for value in values for alpha in alphas]):
        for alpha in alphas:
            print('Alpha = %s' %(alpha))
            data.noreunPhi(event, 'State', values=values, keyword_in='ICA', source_keyword='Clean',
                           keyword_out='alpha=%s' % alpha, alpha=alpha, npoint_art=4,
                           tmin=0.0, tmax=0.3, shared_baseline=False,
                           bl_tmin=-0.4, bl_tmax=-0.01, downsample=False,
                           recalculate_PCI=True, recalculate_baseline=True)

### Plot PCI

In [ ]:
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
values = [value for value in data._default_values('State') if not value == 'Neither']
alphas = [0.01] if event == 'TMS' else [0.01, 0.1, 0.5]
for alpha in alphas:
    data.plotNoreunPCI(event, 'State', values=values, keyword='alpha=%s' %(alpha),
                       evoked_tmin=-0.4, evoked_tmax=0.3)

## Wavelet Connectivity Visualization (not part of pipeline)

In [ ]:
import matplotlib.pyplot as plt

print(data.subject + ' ' + data.session)
event = 'TMS' if data.task == 'TMS-EEG' else 'DBS'
data.waveletConnectivity(event,'State',keyword_in='Clean',
                         keyword_out='WC',downsample=False,
                         bands={'alpha':(8, 15),'beta':(15, 35),
                                'gamma':(35, 150)},tmin=-0.5,tmax=1,
                         time_scalar=2, vis_tmin=-0.1, vis_tmax=0.2,
                         threshold=0.5)
plt.close('all')